<a href="https://colab.research.google.com/github/jiyanshud22/Saltmine-Auto-Stacking/blob/main/BR1_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BR1

#LOGIC



Given:

* A set of **blocks**, each with an area and categorical properties
* A set of **floors**, each with a maximum capacity and area.

We aim to **assign each block** to **one valid floor** such that:

* The block’s **department split** and **space mix category** match that of the floor.
* The block should **fit** (i.e., doesn’t exceed available capacity of the floor)

---

## 🧱 **Input Data Structures**


### ✅ **Blocks Table** (Block Set $\mathcal{B}$)

Each block $b_i \in \mathcal{B}$, for $i = 1 \dots N$, is a tuple:

$$
b_i = \left( A_i,\ D_i,\ C_i \right)
$$

Where:

* $A_i \in \mathbb{R}^{+}$: Area of the block
* $D_i$: Department label
* $C_i$: Space Mix Category

Hence, the full **block matrix** is:

$$
\mathcal{B} =
\begin{bmatrix}
A_1 & D_1 & C_1 \\
A_2 & D_2 & C_2 \\
\vdots & \vdots & \vdots \\
A_N & D_N & C_N
\end{bmatrix}
$$

---

###  **Floors Table** (Floor Set $\mathcal{F}$)

Each floor $f_j \in \mathcal{F}$, for $j = 1 \dots M$, is a tuple:

$$
f_j = \left( \text{Cap}_j,\ D_j,\ C_j,\ U_j \right)
$$

Where:

* $\text{Cap}_j \in \mathbb{R}^{+}$: Total capacity of floor
* $D_j$: Department label
* $C_j$: Space Mix Category
* $U_j$: Used area on floor $j$; initialized to 0

So, the **floor matrix** is:

$$
\mathcal{F} =
\begin{bmatrix}
\text{Cap}_1 & D_1 & C_1 & U_1 \\
\text{Cap}_2 & D_2 & C_2 & U_2 \\
\vdots & \vdots & \vdots & \vdots \\
\text{Cap}_M & D_M & C_M & U_M
\end{bmatrix}
$$

---

## **Matching Logic**

### For each block $b_i$, we search for a floor $f_j \in \mathcal{F}$ such that:

####  **Categorical Match Conditions:**

$$
D_i = D_j \quad \text{(same department)}
$$

$$
C_i = C_j \quad \text{(same space mix category)}
$$

#### **Capacity Constraint check:**

Let $A_i$ be the area of the block and $U_j$ the current used area on floor $j$. Then:

$$
A_i + U_j \leq \text{Cap}_j
$$

If this is satisfied, the block can be placed on that floor.

---

## **Assignment Rule**

Let’s define an **assignment function**:

$$
\phi: \mathcal{B} \rightarrow \mathcal{F} \cup \{\emptyset\}
$$

Such that:

$$
\phi(b_i) = f_j \quad \text{if } D_i = D_j, \ C_i = C_j, \ \text{and } A_i + U_j \leq \text{Cap}_j
$$

If no such floor exists, then:

$$
\phi(b_i) = \emptyset
$$

Once a block $b_i$ is assigned:

$$
U_j \leftarrow U_j + A_i
$$

This ensures the used area of floor $j$ is updated.

---

## **Logical Flow**

###  **Initialize**

* Set all floor used areas $U_j = 0$

### **Block Assignment Loop**

For each block $b_i$:

* Loop through all floors $f_j$
* Check:

  1. $D_i = D_j$
  2. $C_i = C_j$
  3. $A_i + U_j \leq \text{Cap}_j$
* If all true:

  * Assign $\phi(b_i) = f_j$
  * Update $U_j \leftarrow U_j + A_i$
  * Move to next block

---

## 6. **Data Structures as Mathematical Objects**

| Structure         | Math Symbol        | Purpose                                                  |
| ----------------- | ------------------ | -------------------------------------------------------- |
| `floor_df`        | $\mathcal{F}$      | Contains floor capacity, department, category, used area |
| `block_df`        | $\mathcal{B}$      | Contains block area, department, category                |
| `floor_used_area` | $\{ U_j \}$        | Tracks total used area for each floor                    |
| `assigned_blocks` | $\{ (b_i, f_j) \}$ | Stores output block-to-floor assignments                 |

---

## **Final Output**

A mapping:

$$
\{ b_i \rightarrow f_j \mid \phi(b_i) = f_j \neq \emptyset \}
$$

Which produces a **table**:

| Block | Area | Department | Category | Assigned Floor |
| ----- | ---- | ---------- | -------- | -------------- |

---

## Conclusion

we are solving a **discrete matching problem** constrained by:

* **Categorical equality** (Department + Category)
* **Numerical feasibility** (Capacity ≥ Used + New Block)

With each assignment:

* You're **greedily** assigning to the **first matching floor**
* You maintain an internal update of space usage

The entire system is a special case of a **bin packing problem** with constraints.


.




























.





























.


























.








































.

































.

# Code Variables explaied

---

1. **Load & Clean Input Data (Step 1)**

   * Reads an Excel file (`BR-1.xlsx`) with multiple sheets:

     * **Floors**: usable area and capacity for each floor.
     * **Blocks**: individual block definitions, including area and max occupancy.
     * **Department Split**: which sub-departments can be split and their minimum percentages.
     * **Min % Split**: loaded but not used later.
     * **Adjacency**: numeric adjacency constraints between departments.
     * **De-Centralized Logic**: rules for how many extra “destination” floors to use in each mode.
   * Strips whitespace, renames columns to consistent identifiers, and coerces key columns to numeric.

2. **Preprocessing (Step 2)**

   * Splits the block data into **destination** vs. **typical** blocks based on a column flag.

3. **Initialize Floor Assignments (Step 3)**

   * Defines `initialize_floor_assignments()`, which builds a dict for each floor tracking:

     * Remaining area & capacity
     * Assigned blocks & departments
     * Accumulated “space mix” areas for categories ME, WE, US, Support, Speciality

4. **Constraint Helpers (Step 4)**

   * `get_adjacency_constraint(dept1, dept2)`: looks up adjacency weight.
   * `must_be_same_floor()`: returns True if two departments have a positive adjacency requirement.
   * `is_dept_splittable()`: checks whether a department is allowed to span multiple floors.
   * `find_adjacency_groups()`: clusters departments into groups that must co-locate.

5. **Core Stacking Algorithm (Step 5: `run_stack_plan(mode)`)**

   * **Phase 1 (Destination blocks)**

     * Groups destination blocks by “Destination\_Group,” then tries to place each group intact on a limited number of floors (mode-dependent).
     * If a whole group won’t fit, splits at the block level, filling floors greedily by available space.

   * **Phase 2 (Typical blocks)**

     * **2A (Hard constraints)**

       * Identifies “constraint groups” (unsplittable departments and adjacency clusters) and attempts to place each group intact on the floor with enough room.
       * Updates each floor’s space-mix totals as it goes.
     * **2B (Space-mix proportional assignment)**

       * For the remaining (splittable) blocks, buckets them by the five space-mix categories.
       * First assigns all “ME” blocks at random.
       * Computes the fraction of ME blocks per floor, then distributes the other categories (WE, US, Support, Speciality) in proportion to that ME distribution.

   * **Outputs** four Pandas DataFrames:

     1. **detailed\_df**: every assigned block with floor, department, block name, area, occupancy.
     2. **floor\_summary\_df**: per-floor totals (blocks assigned, area, occupancy) merged with original floor capacities.
     3. **space\_mix\_df**: percent of each space-mix category on each floor.
     4. **unassigned\_df**: any blocks that couldn’t be placed.

6. **Optimization Loop & Export (Step 6)**

   * Defines `best_plan(mode, trials)` to run `run_stack_plan` multiple times with different seeds and pick the one with the fewest unassigned blocks.
   * Executes this for three modes: **centralized**, **semi**, **decentralized**.
   * Writes each best plan’s four sheets into its own Excel file:

     * `stack_plan_centralized_constraints.xlsx`
     * `stack_plan_semi_centralized_constraints.xlsx`
     * `stack_plan_decentralized_constraints.xlsx`



In [ ]:


import pandas as pd
import random
import math

# ----------------------------------------
# Step 1: Load All Input Sheets from BR-1.xlsx
# ----------------------------------------

# excel_path: String representing the file path to the input Excel file containing multiple sheets
excel_path = '/content/BR-1.xlsx'  # adjust if needed

# 1.1 Floors sheet
# all_floor_data: DataFrame to store floor-related data from the 'Program Table Input 2 - Floor' sheet
all_floor_data = pd.read_excel(
    excel_path,
    sheet_name='Program Table Input 2 - Floor',
    skiprows=0  # Don't skip header row
)
# Clean column names by stripping whitespace
all_floor_data.columns = all_floor_data.columns.str.strip()
print(all_floor_data.columns.tolist())

# Rename columns for consistency
all_floor_data = all_floor_data.rename(columns={
    all_floor_data.columns[0]: 'Name',  # Name: Floor identifier
    all_floor_data.columns[1]: 'Usable_Area_(SQM)',  # Usable_Area_(SQM): Available space in square meters
    all_floor_data.columns[2]: 'Max_Assignable_Floor_loading_Capacity'  # Max_Assignable_Floor_loading_Capacity: Maximum occupancy limit
})
print(all_floor_data.columns.tolist())

# Ensure numeric columns are properly typed
# Usable_Area_(SQM): Convert to numeric, raising errors for non-numeric values
all_floor_data['Usable_Area_(SQM)'] = pd.to_numeric(
    all_floor_data['Usable_Area_(SQM)'], errors='raise'
)
# Max_Assignable_Floor_loading_Capacity: Convert to numeric, raising errors for non-numeric values
all_floor_data['Max_Assignable_Floor_loading_Capacity'] = pd.to_numeric(
    all_floor_data['Max_Assignable_Floor_loading_Capacity'], errors='raise'
)

# 1.2 Blocks sheet
# all_block_data: DataFrame to store block-related data from the 'Program Table Input 1 - Block' sheet
all_block_data = pd.read_excel(
    excel_path,
    sheet_name='Program Table Input 1 - Block'
)
# Clean column names by stripping whitespace
all_block_data.columns = all_block_data.columns.str.strip()

# Ensure numeric columns are properly typed
# Cumulative_Area_SQM: New column to store block area, converted from Cumulative_Block_Circulation_Area
all_block_data['Cumulative_Area_SQM'] = pd.to_numeric(
    all_block_data['Cumulative_Block_Circulation_Area'], errors='raise'
)
# Max_Occupancy_with_Capacity: Convert to numeric, raising errors for non-numeric values
all_block_data['Max_Occupancy_with_Capacity'] = pd.to_numeric(
    all_block_data['Max_Occupancy_with_Capacity'], errors='raise'
)

# 1.3 Department Split sheet
# department_split_data: DataFrame to store department and sub-department data, skipping the first row
department_split_data = pd.read_excel(
    excel_path,
    sheet_name='Department Split',
    skiprows=1  # Skip the first row which is not the header
)
# Explicitly set column names based on expected structure
department_split_data.columns = [
    'Department_Sub-Department',  # Department_Sub-Department: Department and sub-department identifier
    'Splittable',  # Splittable: Indicates if department blocks can be split (1, 0.75, 0, -1)
    'Min_%_of_Block_per_department'  # Min_%_of_Block_per_department: Minimum percentage of blocks per department
]
# Select only relevant columns
department_split_data = department_split_data[
    ['Department_Sub-Department', 'Splittable', 'Min_%_of_Block_per_department']
].copy()

# Clean column names by stripping whitespace
department_split_data.columns = department_split_data.columns.str.strip()
print(department_split_data.to_string(index=False))

# Build lookup dictionaries
# dept_splittable: Dictionary mapping departments to their splittable value
dept_splittable = department_split_data.set_index('Department_Sub-Department')['Splittable'].to_dict()
# dept_min_pct: Dictionary mapping departments to their minimum percentage of blocks
dept_min_pct = department_split_data.set_index('Department_Sub-Department')['Min_%_of_Block_per_department'].to_dict()

# 1.4 Min%Split sheet
# min_split_data: DataFrame to store data from the 'Min % Split' sheet (not used in logic but loaded)
min_split_data = pd.read_excel(
    excel_path,
    sheet_name='Min % Split'
)
# Clean column names by stripping whitespace
min_split_data.columns = min_split_data.columns.str.strip()

# 1.5 Adjacency sheet
# xls: ExcelFile object to access sheet names in the Excel file
xls = pd.ExcelFile(excel_path)
# adjacency_sheet_name: String containing the name of the sheet that includes "Adjacency"
adjacency_sheet_name = [name for name in xls.sheet_names if "Adjacency" in name][0]
# raw_data: DataFrame containing raw data from the adjacency sheet, with header at row 1 and first column as index
raw_data = xls.parse(adjacency_sheet_name, header=1, index_col=0)
# adjacency_data: DataFrame with numeric values from raw_data, non-numeric values converted to NaN
adjacency_data = raw_data.apply(pd.to_numeric, errors='coerce')
# Clean index and column names by stripping whitespace
adjacency_data.index = adjacency_data.index.str.strip()
adjacency_data.columns = adjacency_data.columns.str.strip()

# 1.6 De-Centralized Logic sheet
# df_logic: DataFrame to store data from the 'De-Centralized Logic' sheet, with no header
df_logic = pd.read_excel(
    excel_path,
    sheet_name='De-Centralized Logic',
    header=None
)
# De_Centralized_data: Dictionary to store logic for centralized, semi-centralized, and decentralized modes
De_Centralized_data = {}
# current_section: String to track the current section being processed (Centralised, Semi Centralized, or DeCentralised)
current_section = None
for _, row in df_logic.iterrows():
    # first_cell: String containing the value of the first column, stripped of whitespace
    first_cell = str(row[0]).strip() if pd.notna(row[0]) else ""
    if first_cell in ["Centralised", "Semi Centralized", "DeCentralised"]:
        # Update current_section to the current mode
        current_section = first_cell
        # Initialize dictionary for the mode with 'Add' key set to 0
        De_Centralized_data[current_section] = {"Add": 0}
    elif current_section and first_cell == "( Add into cetralised destination Block)":
        # Set 'Add' value for the current section to the integer value in the second column
        De_Centralized_data[current_section]["Add"] = int(row[1]) if pd.notna(row[1]) else 0

# Ensure all expected modes are initialized
for key in ["Centralised", "Semi Centralized", "DeCentralised"]:
    if key not in De_Centralized_data:
        # Initialize missing mode with 'Add' set to 0
        De_Centralized_data[key] = {"Add": 0}

# ----------------------------------------
# Step 2: Preprocess Blocks & Department Split
# ----------------------------------------

# 2.2 Separate Destination vs. Typical blocks
# destination_blocks: DataFrame containing blocks marked as 'Destination'
destination_blocks = all_block_data[
    all_block_data['Typical_Destination'] == 'Destination'
].copy()
# typical_blocks: DataFrame containing blocks marked as 'Typical'
typical_blocks = all_block_data[
    all_block_data['Typical_Destination'] == 'Typical'
].copy()

# ----------------------------------------
# Step 3: Initialize Floor Assignments
# ----------------------------------------

def initialize_floor_assignments(floor_df):
    """
    Initializes a dictionary for floor assignments.

    Args:
        floor_df: DataFrame containing floor data with columns 'Name', 'Usable_Area_(SQM)', and 'Max_Assignable_Floor_loading_Capacity'

    Returns:
        assignments: Dictionary keyed by floor name, each entry containing:
            - remaining_area: Float, remaining usable area on the floor
            - remaining_capacity: Integer, remaining occupancy capacity
            - assigned_blocks: List, blocks assigned to the floor
            - assigned_departments: Set, unique departments assigned
            - ME_area, WE_area, US_area, Support_area, Speciality_area: Floats, area for each space mix category
    """
    # assignments: Dictionary to store floor assignment details
    assignments = {}
    for _, row in floor_df.iterrows():
        # floor: String, name of the floor, stripped of whitespace
        floor = row['Name'].strip()
        assignments[floor] = {
            'remaining_area': row['Usable_Area_(SQM)'],  # remaining_area: Initial usable area
            'remaining_capacity': row['Max_Assignable_Floor_loading_Capacity'],  # remaining_capacity: Initial occupancy capacity
            'assigned_blocks': [],  # assigned_blocks: List to store assigned block dictionaries
            'assigned_departments': set(),  # assigned_departments: Set of unique department names
            'ME_area': 0.0,  # ME_area: Area for 'ME' space mix category
            'WE_area': 0.0,  # WE_area: Area for 'WE' space mix category
            'US_area': 0.0,  # US_area: Area for 'US' space mix category
            'Support_area': 0.0,  # Support_area: Area for 'Support' space mix category
            'Speciality_area': 0.0  # Speciality_area: Area for 'Speciality' space mix category
        }
    return assignments

# floors: List of floor names from initialized assignments
floors = list(initialize_floor_assignments(all_floor_data).keys())

# ----------------------------------------
# Step 4: Core Stacking Function (with updated department splitting logic)
# ----------------------------------------

def run_stack_plan(mode):
    """
    Generates a stack plan based on the specified mode.

    Args:
        mode: String, one of 'centralized', 'semi', or 'decentralized'

    Returns:
        detailed_df: DataFrame with block-to-floor assignments
        floor_summary_df: DataFrame with floor totals (count, area, occupancy)
        space_mix_df: DataFrame with space mix distribution per floor
        unassigned_df: DataFrame with unassigned blocks
    """
    # assignments: Dictionary of floor assignments initialized for the current mode
    assignments = initialize_floor_assignments(all_floor_data)
    # unassigned_blocks: List to store blocks that cannot be assigned
    unassigned_blocks = []

    # 4.1 Determine number of floors for destination blocks
    def destination_floor_count():
        # max_dest_floors: Integer, number of floors allocated for destination blocks
        if mode == 'centralized':
            return 2
        elif mode == 'semi':
            return 2 + De_Centralized_data["Semi Centralized"]["Add"]
        elif mode == 'decentralized':
            return 2 + De_Centralized_data["DeCentralised"]["Add"]
        else:
            return 2

    # max_dest_floors: Integer, capped number of floors for destination blocks
    max_dest_floors = min(destination_floor_count(), len(floors))

    # 4.2 Group destination blocks by Destination_Group
    # dest_groups: Dictionary mapping destination group names to their blocks, total area, and capacity
    dest_groups = {}
    for _, blk in destination_blocks.iterrows():
        # grp: String, destination group name
        grp = blk['Destination_Group']
        if grp not in dest_groups:
            dest_groups[grp] = {'blocks': [], 'total_area': 0.0, 'total_capacity': 0}
        # Add block to group's blocks list
        dest_groups[grp]['blocks'].append(blk.to_dict())
        # total_area: Float, cumulative area for the group
        dest_groups[grp]['total_area'] += blk['Cumulative_Area_SQM']
        # total_capacity: Integer, cumulative capacity for the group
        dest_groups[grp]['total_capacity'] += blk['Max_Occupancy_with_Capacity']

    # Phase 1: Assign destination groups
    # group_names: List of destination group names, shuffled for random processing
    group_names = list(dest_groups.keys())
    random.shuffle(group_names)
    for grp in group_names:
        # info_grp: Dictionary containing blocks, total area, and capacity for the group
        info_grp = dest_groups[grp]
        # grp_area: Float, total area required for the group
        grp_area = info_grp['total_area']
        # grp_cap: Integer, total capacity required for the group
        grp_cap = info_grp['total_capacity']
        # placed_whole: Boolean, tracks if the entire group was placed on one floor
        placed_whole = False

        # 4.2.a Attempt to place entire group on any of the first max_dest_floors
        # candidate_floors: List of floor names for destination block placement
        candidate_floors = floors[:max_dest_floors].copy()

        for fl in candidate_floors:
            if (assignments[fl]['remaining_area'] >= grp_area and
                assignments[fl]['remaining_capacity'] >= grp_cap):
                for blk in info_grp['blocks']:
                    assignments[fl]['assigned_blocks'].append(blk)
                    assignments[fl]['assigned_departments'].add(
                        blk['Department_Sub_Department']
                    )
                assignments[fl]['remaining_area'] -= grp_area
                assignments[fl]['remaining_capacity'] -= grp_cap
                placed_whole = True
                break

        # 4.2.b If not placed, try remaining floors
        if not placed_whole:
            for fl in floors[max_dest_floors:]:
                if (assignments[fl]['remaining_area'] >= grp_area and
                    assignments[fl]['remaining_capacity'] >= grp_cap):
                    for blk in info_grp['blocks']:
                        assignments[fl]['assigned_blocks'].append(blk)
                        assignments[fl]['assigned_departments'].add(
                            blk['Department_Sub_Department'].strip()
                        )
                    assignments[fl]['remaining_area'] -= grp_area
                    assignments[fl]['remaining_capacity'] -= grp_cap
                    placed_whole = True
                    break

        # 4.2.c If still not placed, split the group block-by-block
        if not placed_whole:
            # total_remaining_area: Float, total remaining area across all floors
            total_remaining_area = sum(assignments[f]['remaining_area'] for f in floors)
            if total_remaining_area >= grp_area:
                # blocks_sorted: List of blocks sorted by area in descending order
                blocks_sorted = sorted(info_grp['blocks'], key=lambda b: b['Cumulative_Area_SQM'], reverse=True)
                # removed_blocks: List to store blocks removed during trial assignments
                removed_blocks = []
                # trial_blocks: List of blocks for trial group placement
                trial_blocks = blocks_sorted.copy()

                while trial_blocks:
                    # trial_area: Float, total area of trial blocks
                    trial_area = sum(b['Cumulative_Area_SQM'] for b in trial_blocks)
                    # trial_capacity: Integer, total capacity of trial blocks
                    trial_capacity = sum(b['Max_Occupancy_with_Capacity'] for b in trial_blocks)

                    # floor_combination: List of tuples (block, floor) for trial assignments
                    floor_combination = []
                    # temp_assignments: Dictionary, temporary copy of assignments for trial
                    temp_assignments = {f: assignments[f].copy() for f in floors}
                    # temp_floors_by_space: List of floors sorted by remaining area
                    temp_floors_by_space = sorted(floors, key=lambda f: assignments[f]['remaining_area'], reverse=True)

                    # temp_success: Boolean, tracks if trial assignment succeeded
                    temp_success = True
                    for blk in trial_blocks:
                        # blk_area: Float, area of the current block
                        blk_area = blk['Cumulative_Area_SQM']
                        # blk_capacity: Integer, capacity of the current block
                        blk_capacity = blk['Max_Occupancy_with_Capacity']
                        # placed_block: Boolean, tracks if the block was placed
                        placed_block = False

                        for fl in temp_floors_by_space:
                            if (temp_assignments[fl]['remaining_area'] >= blk_area and
                                temp_assignments[fl]['remaining_capacity'] >= blk_capacity):
                                temp_assignments[fl]['remaining_area'] -= blk_area
                                temp_assignments[fl]['remaining_capacity'] -= blk_capacity
                                floor_combination.append((blk, fl))
                                placed_block = True
                                break

                        if not placed_block:
                            temp_success = False
                            break

                    if temp_success:
                        for blk, fl in floor_combination:
                            assignments[fl]['assigned_blocks'].append(blk)
                            assignments[fl]['assigned_departments'].add(blk['Department_Sub_Department'].strip())
                            assignments[fl]['remaining_area'] -= blk['Cumulative_Area_SQM']
                            assignments[fl]['remaining_capacity'] -= blk['Max_Occupancy_with_Capacity']
                        placed_whole = True
                        break
                    else:
                        removed_blocks.append(trial_blocks.pop(0))

                # Place removed blocks individually
                for blk in removed_blocks:
                    # blk_area: Float, area of the block
                    blk_area = blk['Cumulative_Area_SQM']
                    # blk_capacity: Integer, capacity of the block
                    blk_capacity = blk['Max_Occupancy_with_Capacity']
                    # placed_block: Boolean, tracks if the block was placed
                    placed_block = False
                    # floors_by_space: List of floors sorted by remaining area
                    floors_by_space = sorted(floors, key=lambda f: assignments[f]['remaining_area'], reverse=True)

                    for fl in floors_by_space:
                        if (assignments[fl]['remaining_area'] >= blk_area and
                            assignments[fl]['remaining_capacity'] >= blk_capacity):
                            assignments[fl]['assigned_blocks'].append(blk)
                            assignments[fl]['assigned_departments'].add(blk['Department_Sub_Department'].strip())
                            assignments[fl]['remaining_area'] -= blk_area
                            assignments[fl]['remaining_capacity'] -= blk_capacity
                            placed_block = True
                            break

                    if not placed_block:
                        unassigned_blocks.append(blk)
            else:
                for blk in sorted(info_grp['blocks'], key=lambda b: b['Cumulative_Area_SQM'], reverse=True):
                    # blk_area: Float, area of the block
                    blk_area = blk['Cumulative_Area_SQM']
                    # blk_capacity: Integer, capacity of the block
                    blk_capacity = blk['Max_Occupancy_with_Capacity']
                    # placed_block: Boolean, tracks if the block was placed
                    placed_block = False
                    # floors_by_space: List of floors sorted by remaining area
                    floors_by_space = sorted(floors, key=lambda f: assignments[f]['remaining_area'], reverse=True)
                    for fl in floors_by_space:
                        if (assignments[fl]['remaining_area'] >= blk_area and
                            assignments[fl]['remaining_capacity'] >= blk_capacity):
                            assignments[fl]['assigned_blocks'].append(blk)
                            assignments[fl]['assigned_departments'].add(blk['Department_Sub_Department'].strip())
                            assignments[fl]['remaining_area'] -= blk_area
                            assignments[fl]['remaining_capacity'] -= blk_capacity
                            placed_block = True
                            break

                    if not placed_block:
                        unassigned_blocks.append(blk)

    # ----------------------------------------
    # Phase 2: Handle typical blocks with updated department-splittable logic
    # ----------------------------------------

    # 4.3 Categorize typical blocks by department splittable values
    def add_space_mix_area(floor, block, area):
        """Helper function to add area to appropriate space mix category"""
        # cat: String, space mix category of the block
        cat = block['SpaceMix_(ME_WE_US_Support_Speciality)'].strip()
        if cat == 'ME':
            assignments[floor]['ME_area'] += area
        elif cat == 'WE':
            assignments[floor]['WE_area'] += area
        elif cat == 'US':
            assignments[floor]['US_area'] += area
        elif cat.lower() == 'support':
            assignments[floor]['Support_area'] += area
        elif cat.lower() == 'speciality':
            assignments[floor]['Speciality_area'] += area

    # not_splittable_groups: Dictionary, groups blocks with splittable values 1 or 0.75 by department
    not_splittable_groups = {}
    # order_by_area_groups: Dictionary, groups blocks with splittable value 0 by department
    order_by_area_groups = {}
    # fully_splittable_blocks: List, blocks with splittable value -1
    fully_splittable_blocks = []

    for blk in typical_blocks.to_dict('records'):
        # dept: String, department/sub-department of the block
        dept = blk['Department_Sub_Department'].strip()
        # spl: Float or Integer, splittable value for the department, defaulting to 1
        spl = dept_splittable.get(dept, 1)

        if spl in [1, 0.75]:
            not_splittable_groups.setdefault(dept, []).append(blk)
        elif spl == 0:
            order_by_area_groups.setdefault(dept, []).append(blk)
        else:
            fully_splittable_blocks.append(blk)

    # 4.4 Phase 2A: Assign not_splittable departments
    print(f"Processing {len(not_splittable_groups)} not-splittable departments...")
    for dept, blocks_list in not_splittable_groups.items():
        # total_area: Float, total area of blocks in the department
        total_area = sum(b['Cumulative_Area_SQM'] for b in blocks_list)
        # total_cap: Integer, total capacity of blocks in the department
        total_cap = sum(b['Max_Occupancy_with_Capacity'] for b in blocks_list)
        # placed: Boolean, tracks if the department was placed
        placed = False

        # candidate_floors: List of floors sorted by remaining area
        candidate_floors = sorted(
            floors,
            key=lambda f: assignments[f]['remaining_area'],
            reverse=True
        )
        for fl in candidate_floors:
            if (assignments[fl]['remaining_area'] >= total_area and
                assignments[fl]['remaining_capacity'] >= total_cap):
                for blk in blocks_list:
                    assignments[fl]['assigned_blocks'].append(blk)
                    assignments[fl]['assigned_departments'].add(dept)
                    add_space_mix_area(fl, blk, blk['Cumulative_Area_SQM'])
                assignments[fl]['remaining_area'] -= total_area
                assignments[fl]['remaining_capacity'] -= total_cap
                placed = True
                print(f"  Placed dept {dept} on floor {fl}")
                break

        if not placed:
            unassigned_blocks.extend(blocks_list)
            print(f"  Could not place dept {dept} - added to unassigned")

    # 4.5 Phase 2B: Assign order_by_area departments
    print(f"Processing {len(order_by_area_groups)} order-by-area departments...")
    for dept, blocks_list in order_by_area_groups.items():
        # total_area: Float, total area of blocks in the department
        total_area = sum(b['Cumulative_Area_SQM'] for b in blocks_list)
        # total_cap: Integer, total capacity of blocks in the department
        total_cap = sum(b['Max_Occupancy_with_Capacity'] for b in blocks_list)

        # placed_whole: Boolean, tracks if the entire department was placed on one floor
        placed_whole = False
        # floors_by_remaining_area: List of floors sorted by remaining area
        floors_by_remaining_area = sorted(
            floors,
            key=lambda f: assignments[f]['remaining_area'],
            reverse=True
        )

        for fl in floors_by_remaining_area:
            if (assignments[fl]['remaining_area'] >= total_area and
                assignments[fl]['remaining_capacity'] >= total_cap):
                for blk in blocks_list:
                    assignments[fl]['assigned_blocks'].append(blk)
                    assignments[fl]['assigned_departments'].add(dept)
                    add_space_mix_area(fl, blk, blk['Cumulative_Area_SQM'])
                assignments[fl]['remaining_area'] -= total_area
                assignments[fl]['remaining_capacity'] -= total_cap
                placed_whole = True
                print(f"  Placed dept {dept} entirely on floor {fl}")
                break

        if not placed_whole:
            print(f"  Splitting dept {dept} across floors...")
            for blk in blocks_list:
                # blk_area: Float, area of the current block
                blk_area = blk['Cumulative_Area_SQM']
                # blk_capacity: Integer, capacity of the current block
                blk_capacity = blk['Max_Occupancy_with_Capacity']
                # placed_block: Boolean, tracks if the block was placed
                placed_block = False

                floors_by_remaining_area = sorted(
                    floors,
                    key=lambda f: assignments[f]['remaining_area'],
                    reverse=True
                )

                for fl in floors_by_remaining_area:
                    if (assignments[fl]['remaining_area'] >= blk_area and
                        assignments[fl]['remaining_capacity'] >= blk_capacity):
                        assignments[fl]['assigned_blocks'].append(blk)
                        assignments[fl]['assigned_departments'].add(dept)
                        add_space_mix_area(fl, blk, blk_area)
                        assignments[fl]['remaining_area'] -= blk_area
                        assignments[fl]['remaining_capacity'] -= blk_capacity
                        placed_block = True
                        break

                if not placed_block:
                    unassigned_blocks.append(blk)

    # 4.6 Phase 2C: Assign fully_splittable blocks
    print(f"Processing {len(fully_splittable_blocks)} fully-splittable blocks...")

    # 4.6.a Assign ME blocks randomly
    # me_blocks: List of blocks with 'ME' space mix category
    me_blocks = [
        blk for blk in fully_splittable_blocks
        if blk['SpaceMix_(ME_WE_US_Support_Speciality)'].strip() == 'ME'
    ]
    random.shuffle(me_blocks)
    for blk in me_blocks:
        # blk_area: Float, area of the block
        blk_area = blk['Cumulative_Area_SQM']
        # blk_capacity: Integer, capacity of the block
        blk_capacity = blk['Max_Occupancy_with_Capacity']
        # blk_dept: String, department of the block
        blk_dept = blk['Department_Sub_Department'].strip()

        # candidate_floors: List of floors for random assignment
        candidate_floors = floors.copy()
        random.shuffle(candidate_floors)
        # placed: Boolean, tracks if the block was placed
        placed = False
        for fl in candidate_floors:
            if (assignments[fl]['remaining_area'] >= blk_area and
                assignments[fl]['remaining_capacity'] >= blk_capacity):
                assignments[fl]['assigned_blocks'].append(blk)
                assignments[fl]['remaining_area'] -= blk_area
                assignments[fl]['remaining_capacity'] -= blk_capacity
                assignments[fl]['assigned_departments'].add(blk_dept)
                assignments[fl]['ME_area'] += blk_area
                placed = True
                break
        if not placed:
            unassigned_blocks.append(blk)

    # 4.6.b Compute ME distribution per floor
    # me_count_per_floor: Dictionary, count of ME blocks per floor
    me_count_per_floor = {fl: 0 for fl in floors}
    for fl, info in assignments.items():
        me_count_per_floor[fl] = sum(
            1 for blk in info['assigned_blocks']
            if blk['SpaceMix_(ME_WE_US_Support_Speciality)'].strip() == 'ME'
        )
    # total_me: Integer, total number of ME blocks assigned
    total_me = sum(me_count_per_floor.values())
    # me_frac_per_floor: Dictionary, fraction of ME blocks per floor
    if total_me == 0:
        me_frac_per_floor = {fl: 1 / len(floors) for fl in floors}
    else:
        me_frac_per_floor = {
            fl: me_count_per_floor[fl] / total_me for fl in floors
        }

    # 4.6.c Assign other categories proportionally
    # other_categories: List of non-ME space mix categories
    other_categories = ['WE', 'US', 'Support', 'Speciality']
    for category in other_categories:
        # cat_blocks: List of blocks for the current category
        cat_blocks = [
            blk for blk in fully_splittable_blocks
            if blk['SpaceMix_(ME_WE_US_Support_Speciality)'].strip() == category
        ]
        # total_cat: Integer, total number of blocks in the category
        total_cat = len(cat_blocks)
        if total_cat == 0:
            continue

        # raw_targets: Dictionary, raw target counts per floor based on ME distribution
        raw_targets = {fl: me_frac_per_floor[fl] * total_cat for fl in floors}
        # target_counts: Dictionary, rounded target counts per floor
        target_counts = {fl: int(round(raw_targets[fl])) for fl in floors}

        # diff: Integer, difference between total blocks and sum of target counts
        diff = total_cat - sum(target_counts.values())
        if diff != 0:
            # fractional_parts: Dictionary, fractional parts of raw counts for adjustment
            fractional_parts = {
                fl: raw_targets[fl] - math.floor(raw_targets[fl]) for fl in floors
            }
            if diff > 0:
                for fl in sorted(floors, key=lambda x: fractional_parts[x], reverse=True)[:diff]:
                    target_counts[fl] += 1
            else:
                for fl in sorted(floors, key=lambda x: fractional_parts[x])[: -diff]:
                    target_counts[fl] -= 1

        random.shuffle(cat_blocks)
        # assigned_counts: Dictionary, tracks assigned blocks per floor
        assigned_counts = {fl: 0 for fl in floors}

        for blk in cat_blocks:
            # blk_area: Float, area of the block
            blk_area = blk['Cumulative_Area_SQM']
            # blk_capacity: Integer, capacity of the block
            blk_capacity = blk['Max_Occupancy_with_Capacity']
            # blk_dept: String, department of the block
            blk_dept = blk['Department_Sub_Department'].strip()

            # deficits: Dictionary, remaining blocks needed to meet target counts
            deficits = {fl: target_counts[fl] - assigned_counts[fl] for fl in floors}
            # floors_with_deficit: List of floors with remaining target counts
            floors_with_deficit = [fl for fl, d in deficits.items() if d > 0]
            if floors_with_deficit:
                # candidate_floors: List of floors sorted by deficit
                candidate_floors = sorted(
                    floors_with_deficit,
                    key=lambda x: deficits[x],
                    reverse=True
                )
            else:
                candidate_floors = floors.copy()

            # placed: Boolean, tracks if the block was placed
            placed = False
            for fl in candidate_floors:
                if (assignments[fl]['remaining_area'] >= blk_area and
                    assignments[fl]['remaining_capacity'] >= blk_capacity):
                    assignments[fl]['assigned_blocks'].append(blk)
                    assignments[fl]['remaining_area'] -= blk_area
                    assignments[fl]['remaining_capacity'] -= blk_capacity
                    assignments[fl]['assigned_departments'].add(blk_dept)
                    add_space_mix_area(fl, blk, blk_area)
                    assigned_counts[fl] += 1
                    placed = True
                    break

            if not placed:
                # fallback: List of floors for random assignment
                fallback = floors.copy()
                random.shuffle(fallback)
                for fl in fallback:
                    if (assignments[fl]['remaining_area'] >= blk_area and
                        assignments[fl]['remaining_capacity'] >= blk_capacity):
                        assignments[fl]['assigned_blocks'].append(blk)
                        assignments[fl]['remaining_area'] -= blk_area
                        assignments[fl]['remaining_capacity'] -= blk_capacity
                        assignments[fl]['assigned_departments'].add(blk_dept)
                        add_space_mix_area(fl, blk, blk_area)
                        assigned_counts[fl] += 1
                        placed = True
                        break

            if not placed:
                unassigned_blocks.append(blk)

    # Final pass: Try to place remaining unassigned blocks
    for blk in unassigned_blocks.copy():
        # ba: Float, area of the block
        ba = blk['Cumulative_Area_SQM']
        # bc: Integer, capacity of the block
        bc = blk['Max_Occupancy_with_Capacity']
        for fl in floors:
            if (assignments[fl]['remaining_area'] >= ba and
                assignments[fl]['remaining_capacity'] >= bc):
                assignments[fl]['assigned_blocks'].append(blk)
                assignments[fl]['assigned_departments'].add(
                    blk['Department_Sub_Department'].strip()
                )
                assignments[fl]['remaining_area'] -= ba
                assignments[fl]['remaining_capacity'] -= bc
                unassigned_blocks.remove(blk)
                break

    # 4.6 Phase 3: Build Output DataFrames

    # 4.6.1 Detailed DataFrame
    # assignment_list: List of dictionaries containing block assignment details
    assignment_list = []
    for fl, info in assignments.items():
        for blk in info['assigned_blocks']:
            assignment_list.append({
                'Floor': fl,  # Floor: Name of the floor
                'Department': blk['Department_Sub_Department'],  # Department: Department/sub-department
                'Block_Name': blk['Block_Name'],  # Block_Name: Name of the block
                'Destination_Group': blk['Destination_Group'],  # Destination_Group: Group identifier
                'SpaceMix': blk['SpaceMix_(ME_WE_US_Support_Speciality)'],  # SpaceMix: Space mix category
                'Assigned_Area_SQM': blk['Cumulative_Area_SQM'],  # Assigned_Area_SQM: Assigned area
                'Max_Occupancy': blk['Max_Occupancy_with_Capacity']  # Max_Occupancy: Occupancy capacity
            })
    # detailed_df: DataFrame with block assignments
    detailed_df = pd.DataFrame(assignment_list)

    # 4.6.2 Floor_Summary DataFrame
    # floor_summary_df: DataFrame aggregating block count, area, and occupancy per floor
    floor_summary_df = (
        detailed_df
        .groupby('Floor')
        .agg(
            Assgn_Blocks=('Block_Name', 'count'),  # Assgn_Blocks: Number of blocks per floor
            Assgn_Area_SQM=('Assigned_Area_SQM', 'sum'),  # Assgn_Area_SQM: Total area per floor
            Total_Occupancy=('Max_Occupancy', 'sum')  # Total_Occupancy: Total occupancy per floor
        )
        .reset_index()
    )

    # floor_input_subset: DataFrame with original floor data for merging
    floor_input_subset = all_floor_data[[
        'Name', 'Usable_Area_(SQM)', 'Max_Assignable_Floor_loading_Capacity'
    ]].rename(columns={
        'Name': 'Floor',  # Floor: Floor name
        'Usable_Area_(SQM)': 'Input_Usable_Area_SQM',  # Input_Usable_Area_SQM: Original area
        'Max_Assignable_Floor_loading_Capacity': 'Input_Max_Capacity'  # Input_Max_Capacity: Original capacity
    })

    # Merge input data with summary
    floor_summary_df = pd.merge(
        floor_input_subset,
        floor_summary_df,
        on='Floor',
        how='left'
    )

    # Fill NaNs for floors with no assignments
    floor_summary_df[[
        'Assgn_Blocks',
        'Assgn_Area_SQM',
        'Total_Occupancy'
    ]] = floor_summary_df[[
        'Assgn_Blocks',
        'Assgn_Area_SQM',
        'Total_Occupancy'
    ]].fillna(0)

    # 4.6.3 SpaceMix_By_Units DataFrame
    # all_categories: List of space mix categories
    all_categories = ['ME', 'WE', 'US', 'Support', 'Speciality']
    # category_totals: Dictionary, total count of blocks per category
    category_totals = {
        cat: len(typical_blocks[
            typical_blocks['SpaceMix_(ME_WE_US_Support_Speciality)'].str.strip() == cat
        ])
        for cat in all_categories
    }

    # rows: List of dictionaries for space mix data
    rows = []
    for fl, info in assignments.items():
        # counts: Dictionary, count of blocks per category on the floor
        counts = {cat: 0 for cat in all_categories}
        for blk in info['assigned_blocks']:
            # cat: String, space mix category of the block
            cat = blk['SpaceMix_(ME_WE_US_Support_Speciality)'].strip()
            if cat in counts:
                counts[cat] += 1
        # total_blocks_on_floor: Integer, total blocks on the floor
        total_blocks_on_floor = sum(counts.values())

        for cat in all_categories:
            # cnt: Integer, count of blocks for the category
            cnt = counts[cat]
            # pct_of_floor: Float, percentage of floor's blocks for the category
            pct_of_floor = (cnt / total_blocks_on_floor * 100) if total_blocks_on_floor else 0.0
            # total_cat: Integer, total blocks in the category
            total_cat = category_totals[cat]
            # pct_overall: Float, percentage of category's blocks on this floor
            pct_overall = (cnt / total_cat * 100) if total_cat else 0.0

            rows.append({
                'Floor': fl,
                'SpaceMix': cat,
                '%spaceMix': round(pct_overall, 2)  # %spaceMix: Percentage of category's blocks
            })

    # space_mix_df: DataFrame with space mix distribution
    space_mix_df = pd.DataFrame(rows)

    # 4.6.4 Unassigned DataFrame
    # unassigned_list: List of dictionaries for unassigned blocks
    unassigned_list = []
    for blk in unassigned_blocks:
        unassigned_list.append({
            'Department': blk.get('Department_Sub_Department', ''),  # Department: Department/sub-department
            'Block_Name': blk.get('Block_Name', ''),  # Block_Name: Name of the block
            'Destination_Group': blk.get('Destination_Group', ''),  # Destination_Group: Group identifier
            'SpaceMix': blk.get('SpaceMix_(ME_WE_US_Support_Speciality)', ''),  # SpaceMix: Category
            'Area_SQM': blk.get('Cumulative_Area_SQM', 0),  # Area_SQM: Block area
            'Max_Occupancy': blk.get('Max_Occupancy_with_Capacity', 0)  # Max_Occupancy: Block capacity
        })
    # unassigned_df: DataFrame with unassigned blocks
    unassigned_df = pd.DataFrame(unassigned_list)

    return detailed_df, floor_summary_df, space_mix_df, unassigned_df

# ----------------------------------------
# Step 5: Run & Export (with sampling to minimize unassigned)
# ----------------------------------------

def best_plan(mode, trials=50):
    """
    Runs multiple trials to find the plan with the fewest unassigned blocks.

    Args:
        mode: String, one of 'centralized', 'semi', or 'decentralized'
        trials: Integer, number of trials to run (default 50)

    Returns:
        Tuple of DataFrames: detailed_df, floor_summary_df, space_mix_df, unassigned_df
    """
    # best: Tuple, stores the best plan found
    best = None
    # best_unassigned: Integer, tracks the smallest number of unassigned blocks
    best_unassigned = float('inf')
    for seed in range(trials):
        random.seed(seed)
        # det, fs, sm, un: DataFrames returned by run_stack_plan
        det, fs, sm, un = run_stack_plan(mode)
        if len(un) < best_unassigned:
            best_unassigned = len(un)
            best = (det, fs, sm, un)
    return best

# Run stack plans for each mode
# central_detailed, central_floor_sum, central_space_mix, central_unassigned: DataFrames for centralized mode
central_detailed, central_floor_sum, central_space_mix, central_unassigned = best_plan('centralized', trials=50)
# semi_detailed, semi_floor_sum, semi_space_mix, semi_unassigned: DataFrames for semi-centralized mode
semi_detailed, semi_floor_sum, semi_space_mix, semi_unassigned = best_plan('semi', trials=50)
# decentral_detailed, decentral_floor_sum, decentral_space_mix, decentral_unassigned: DataFrames for decentralized mode
decentral_detailed, decentral_floor_sum, decentral_space_mix, decentral_unassigned = best_plan('decentralized', trials=50)

# File names for output Excel files
# central_file: String, filename for centralized mode output
central_file = 'stack_plan_centralized28.xlsx'
# semi_file: String, filename for semi-centralized mode output
semi_file = 'stack_plan_semi_centralized28.xlsx'
# decentral_file: String, filename for decentralized mode output
decentral_file = 'stack_plan_decentralized28.xlsx'

# Export to Excel
with pd.ExcelWriter(central_file) as writer:
    central_detailed.to_excel(writer, sheet_name='Detailed', index=False)
    central_floor_sum.to_excel(writer, sheet_name='Floor_Summary', index=False)
    central_space_mix.to_excel(writer, sheet_name='SpaceMix_By_Units', index=False)
    central_unassigned.to_excel(writer, sheet_name='Unassigned', index=False)

with pd.ExcelWriter(semi_file) as writer:
    semi_detailed.to_excel(writer, sheet_name='Detailed', index=False)
    semi_floor_sum.to_excel(writer, sheet_name='Floor_Summary', index=False)
    semi_space_mix.to_excel(writer, sheet_name='SpaceMix_By_Units', index=False)
    semi_unassigned.to_excel(writer, sheet_name='Unassigned', index=False)

with pd.ExcelWriter(decentral_file) as writer:
    decentral_detailed.to_excel(writer, sheet_name='Detailed', index=False)
    decentral_floor_sum.to_excel(writer, sheet_name='Floor_Summary', index=False)
    decentral_space_mix.to_excel(writer, sheet_name='SpaceMix_By_Units', index=False)
    decentral_unassigned.to_excel(writer, sheet_name='Unassigned', index=False)

print("✅ Generated three Excel outputs (each with an 'Unassigned' sheet):")
print(f"    • {central_file}")
print(f"    • {semi_file}")
print(f"    • {decentral_file}")


Streaming output truncated to the last 5000 lines.
  Could not place dept Common_Common_Common - added to unassigned
  Placed dept External_External_External on floor L012Level 12
Processing 0 order-by-area departments...
Processing 152 fully-splittable blocks...
Processing 49 not-splittable departments...
  Placed dept PUBLIC SPACE_FRONT OFFICE_CAO/SC BM on floor L012Level 12
  Placed dept PUBLIC SPACE_FRONT OFFICE_Cabin on floor L012Level 12
  Placed dept PUBLIC SPACE_FRONT OFFICE_GS on floor L012Level 12
  Placed dept PUBLIC SPACE_FRONT OFFICE_CAO (DBS/DInning) on floor L012Level 12
  Placed dept PUBLIC SPACE_FRONT OFFICE_CB on floor L012Level 12
  Placed dept PUBLIC SPACE_FRONT OFFICE_Chief Inv officer (TCIO) on floor L012Level 12
  Placed dept PUBLIC SPACE_FRONT OFFICE_CIB on floor L012Level 12
  Placed dept PUBLIC SPACE_FRONT OFFICE_Depository Service on floor L012Level 12
  Placed dept PUBLIC SPACE_FRONT OFFICE_Escrow on floor L012Level 12
  Placed dept PUBLIC SPACE_FRONT OFFICE